In [2]:
import os
%pwd

'e:\\textsummarizer\\research'

In [3]:
os.chdir('../')

In [4]:
from dataclasses import  dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir:Path
    data_path: Path
    tokenizer_name:Path
    

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml,create_directories

In [6]:
 class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_path)
        self.paramss=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self)-> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config=DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config
        

In [7]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

e:\textsummarizer\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Transformation Component

In [8]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):

        input_encodings = self.tokenizer(
            example_batch['dialogue'],
            max_length=1024,
            truncation=True
        )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(
                example_batch['summary'],
                max_length=128,
                truncation=True
            )

        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    def convert(self):
         dataset_samsum=load_from_disk(self.config.data_path)

         dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True, num_proc=1)
         dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [9]:
config=ConfigurationManager()
data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config=data_transformation_config)
data_transformation.convert()

2026-02-27 18:29:50,253 - INFO - common - yaml file:config\config.yaml loaded successfully
2026-02-27 18:29:50,257 - INFO - common - yaml file:params.yaml loaded successfully
2026-02-27 18:29:50,259 - INFO - common - created directory at:artifacts
2026-02-27 18:29:50,261 - INFO - common - created directory at:artifacts/data_transformation
2026-02-27 18:29:51,126 - INFO - _client - HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"
2026-02-27 18:29:51,253 - INFO - _client - HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"
2026-02-27 18:29:51,558 - INFO - _client - HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"
2026-02-27 18:29:51,600 - INFO - _client - HTTP Request: HEAD https://huggingface.co/api/reso

Map (num_proc=1):   0%|          | 0/14732 [00:09<?, ? examples/s]


AttributeError: PegasusTokenizer has no attribute as_target_tokenizer